### 나는 이 연구를 왜 이 시점에 진행하는가?
 소비자행동론 강의에서 시험 대신  Term-paper를 작성해서 12월 9일까지 제출해야하기 때문이다. 그러나 단지 과제용에 국한되지 않고 이후 발전시켜 최종 논문 작성을 목표로 한다. 

---
### 연구 주제는 무엇인가? 
가제: 온라인 쇼핑몰 소비자의 정보탐색 행동 대한 실증적 분석: 롯데멤버십 회원을 중심으로

---
### 연구 주제 선정 배경 
개인적인 관점에서는 객관적인 로그 중심의 온라인 소비행동이 재밌다. 온라인 행동 데이터를 기반으로 소비자 및 사용자를 추적하는 과정이 형사 내지는 검사가 된 느낌이랄까

산업적인 관점에서 바라보자면 PC와 스마트폰, 인터넷의 보급과 함께 E-commerce의 성장을 놀라운 추세였으나 최근 COVID-19로 그 흐름이 더 가속화되고 있다. 또한 공급의 상향 평준화로 품질만의 속성을 통해 소비자를 설득시키는 것이 한계점에 다다랐다. 따라서 기업들은(특히 유통업) 이전까지 행해지던 소비자 집단에 대한 분석이 아닌 소비자 개인에 대한 분석을 통해 얻은 인사이트로 '맞춤화'를 실현하여 소비를 이끌어내고 있다. 
온라인에서의 행동은 오프라인에서와 달리 기록이 더 쉬우며 연속형 데이터로서 측정되기 때문에 마케터 관점에서 통계적 분석을 통해 소비자에 대한 인사이트를 찾는 데에 용이한 측면이 있다. 실제로 아마존, 페이스북, 유튜브 비롯하여 국내 기업들, 네이버, 쿠팡, 롯데 등에서도 회원의 개인정보와 Hit, Session 지속 시간, 검색어, 페이지 뷰 수 등 온라인에서의 행동 데이터를 분석하여 소비자에 대한 인사이트를 찾고 있다. 

마케팅을 공부하는 연구자로서 학술적인 관점에서 바라보자면, 온라인에서 소비가 이루어진다는 것은 소비환경이 변했음을 의미한다. E-commerce와 온라인 소비의 성장세는 단지 COVID-19와 같은 사태로 일어난 일시적인 현상이 아니라 거부할 수 없는 흐름이다. 따라서 이전까지 소비자행동론의 분석들은 대부분 오프라인 소비 환경 위에서 정립된 이론들을 변화한 소비 환경 패러다임 속에서 다시 연구하고 재정의할 필요성이 있다. 

물론 소비자행동론에서 통용되는 이론들은 국가, 인종, 문화, 온오프라인 등 다양한 소비자를 둘러싼 다양한 환경을 불문하고 보편적으로 적용되는 이론들이 많다. 하지만 한 기업의 입장에서 그 기업이 주 타깃으로 하는, 또는 주로 마주하게 되는 소비자들에 대한 실증적 연구는 부족한 실정이다.  

또한 대부분의 온라인 소비 행동은 EDA (Exploratory Data Analysis)가 많이 이루어졌다. 탐색적자료분석은 데이터를 기술적(technical)으로 탐색하고 데이터의 특징과 구조적인 측면을 파악하여 통찰(insight)를 얻는 것이다. 
~그러나 EDA는 이러한 문제점을 안고 있으니 CDA를 통해 그런 점을 보완하고자 한다. 소위말하는 빅데이터셋을 대상으로, 또 온라인 소비행동 데이터를 그렇게 CDA로 진행한 연구는 부족하다. 

---
### 왜 연구에 Lpoint 데이터를 사용했는가? 
물론 교수님께서는 실험계획법(Design of Experiments)에 대한 학습을 목적으로 분석 방법을 실험계획법의 분석방법론을 사용하라고 하셨다. 실험계획법은 계획->데이터 확보 -> 분석의 순서를 따르는 확증적 데이터 분석(Confirmatory Data Analysis) 중 하나이다. 그러나 나는 
- 자기기입식 설문조사를 크게 신뢰하지 않는다.
- 타당성을 조금이라도 높이기 위해 객관적으로 기록된 데이터를 사용하고 싶다.
- EDA로 인사이트와 가설을 도출해내고 CDA로 검정 또는 CDA로 오히려 가설을 도출해낼 수 있다고 판단한다.

따라서 2018년 공모전 참가에서 활용했던 Lpoint 데이터셋(비록 완전 진짜 데이터는 아니지만)을 활용하여 CD 

---
### 분석 방법 



---
### 예상되는 결과



---
### 본 연구를 통해 얻을 수 있는 인사이트 (연구의 가치)


In [ ]:
#구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#출력 한 번에 나올 수 있게 설정
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
filename='/content/drive/My Drive/DataSet/Lpoint/data/Product.csv'

In [ ]:
prd = pd.read_csv(filename, dtype={"CLNT_ID":np.str, "SESS_ID":np.str},thousands=",") #파일 불러오기
prd.head()
prd.dtypes
prd.shape
#행이 5024906, Product가 구매 이력 데이터이기 때문에 행의 수가 곧 6개월 간 구매빈도

In [ ]:
mastercsv='/content/drive/My Drive/DataSet/Lpoint/data/Master.csv'
master = pd.read_csv(mastercsv)

customcsv='/content/drive/My Drive/DataSet/Lpoint/data/Custom.csv'
custom = pd.read_csv(customcsv, dtype={"CLNT_ID": np.str})

sessioncsv='/content/drive/My Drive/DataSet/Lpoint/data/Session.csv'
session = pd.read_csv(sessioncsv, dtype={"CLNT_ID": np.str, "SESS_ID":np.str}, thousands=",")

srchcsv='/content/drive/My Drive/DataSet/Lpoint/data/Search1.csv'
search1 = pd.read_csv(srchcsv, dtype={"CLNT_ID":np.str, "SESS_ID":np.str})

srch2csv='/content/drive/My Drive/DataSet/Lpoint/data/Search2.csv'
search2 = pd.read_csv(srch2csv)

## Preprocessing
### 불러온 각 데이터셋 별 전처리 필요한 항목
① Product 
- HITS_SEQ 1~2인 경우, 일반적으로 클릭 한 두 번에 구매가 가능하지 않다. Session과 병합 이후 살펴볼 예정

② Custorm 
- 없음 (마찬가지로 다른 데이터셋 전처리와 병합 이후 필요한 부분에 있어서 진행할 예정)

③ Master 
- 대분류 항목 간소화 해야함. 내구재vs비내구재, 의류vs비의류와 같이 특정 연구 목적에 맞게 분류를 조정할 필요가 있음 

④ Session 
- TOT_SESS_HR_V 말도 안되게 작은 것들 처리해야함. 그러나 그 기준을 어떻게 산정해야할지 문제. 
